<a href="https://colab.research.google.com/github/vbejaranos/Data-Science_Habi/blob/main/Habi_Concurso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Modulos y librerias**




In [ ]:
#!pip install geopandas
#!pip install pyshp

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
# Trabajo con tablas y arreglos ==================================================================================#
import pandas as pd
import numpy as np
from math import sqrt
# Librerias para graficos ========================================================================================#
import matplotlib as plt
from matplotlib.pylab import rcParams
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy import stats
def _color_red_or_green(val):
    color = 'red' if np.abs(val) < 0.4 else 'green'
    return 'color: %s' % color
# Librerias para georeferencia ===================================================================================#
#https://geopandas.org/docs/user_guide/set_operations.html
#from shapely.geometry import Polygon
#import geopandas
#import shapefile
#import fiona
# Librerias para el modelamiento =================================================================================#
import sklearn as skl
from sklearn.linear_model import LinearRegression # Regresión lineal
from sklearn.svm import SVR # Support Vector Regression
from sklearn.neural_network import MLPRegressor # Redes Neuronales
from sklearn.datasets import make_regression # Redes Neuronales
from sklearn.impute import KNNImputer # Imputacion knn
# Librerías para validación de modelos ===========================================================================#
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
def starMV(x): return 0.0001 if x<1 else x
from sklearn.model_selection import cross_val_score
#Guardar resultados ===============================================================================================# 
import joblib
import os
#from google.colab import files
#uploaded = files.upload()

## Conexión Google Drive con Colab 

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


## Bases de datos completas
Una vez se realizó la intersección con los multipoligonos dados por [catastro](https://mapas.bogota.gov.co/?l=10462&e=-74.15372037610308,4.633364461430024,-74.0713229151657,4.674812311999409,4686&b=7256#) 

In [ ]:
data_train = pd.read_csv('/content/gdrive/MyDrive/shp/train_dataV02.csv')
data_test = pd.read_csv('/content/gdrive/MyDrive/shp/test_dataV02.csv')

### Depuración base de datos



In [ ]:
#banoservicio NA = 0
#ascensor NA = 0
#balcon NA = 0
#cuartoservicio NA = 0
#gimnasio NA = 0
#piscina NA = 0
#terraza NA = 0
#remodelado NA = 0

#var tipoinmueble
#deposito apto = 0, casa = 1
#estudio apto = 0, casa = 1
#porteria conjunto cerrado = 1, no cerrado = 0
#zonalavanderia casas = 1, conjunto residencial_1 = knn, resto

#hall de alcobas condicional al numero de cuartos >1
#parqueaderovisitantes condicional al numero de garajes>1 = garajes - 1

In [ ]:
data_train[['tipo_data']]= "train"
data_test[['tipo_data']] = "test"
consol = data_train.append(data_test)

In [ ]:
consol[['banoservicio', 'ascensor', 'balcon', 'cuartoservicio', 'gimnasio', 'piscina', 'terraza', 'remodelado']] = consol[['banoservicio', 'ascensor', 'balcon', 'cuartoservicio', 'gimnasio', 'piscina', 'terraza', 'remodelado']].fillna(0)

In [ ]:
consol.columns

Index(['Unnamed: 0', 'id', 'area', 'ascensor', 'balcon', 'banos',
       'banoservicio', 'conjuntocerrado', 'cuartoservicio', 'deposito',
       'estrato', 'estudio', 'garajes', 'gimnasio', 'habitaciones',
       'halldealcobas', 'parqueaderovisitantes', 'piscina', 'piso', 'porteria',
       'remodelado', 'saloncomunal', 'terraza', 'tiempodeconstruido',
       'vigilancia', 'vista', 'zonalavanderia', 'valoradministracion',
       'valorventa', 'tipoinmueble', 'tiponegocio', 'latitud', 'longitud',
       'id2', 'MANCODIGO', 'V_REF', 'ANO', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry', 'Ano2', 'tipo_data'],
      dtype='object')

In [ ]:
consol['tiponegocio'] = consol['tiponegocio'].str.lower()
consol

,Unnamed: 0,id,area,ascensor,balcon,banos,banoservicio,conjuntocerrado,cuartoservicio,deposito,estrato,estudio,garajes,gimnasio,habitaciones,halldealcobas,parqueaderovisitantes,piscina,piso,porteria,remodelado,saloncomunal,terraza,tiempodeconstruido,vigilancia,vista,zonalavanderia,valoradministracion,valorventa,tipoinmueble,tiponegocio,latitud,longitud,id2,MANCODIGO,V_REF,ANO,SHAPE_AREA,SHAPE_LEN,geometry,Ano2,tipo_data
0,0,1205404,55.0,0.0,0.0,NaN,0.0,0,0.0,1.0,3.0,NaN,NaN,0.0,2.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1.0,50000.0,1.699500e+08,Casa,venta,4.560965,-74.094889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
1,1,1204371,43.0,0.0,0.0,NaN,0.0,0,0.0,0.0,NaN,NaN,NaN,0.0,3.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,9.000000e+07,Casa,venta,4.576960,-74.100083,382367.0,1401036.0,1800000.0,2020-01-01,3.082467e-07,0.002643,"POLYGON ((-74.099872151 4.576670392999972, -74...",2020.0,train
2,2,1125353,360.0,0.0,0.0,NaN,0.0,0,0.0,0.0,3.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.100000e+09,Casa,venta,4.575000,-74.101000,363088.0,1401044.0,1850000.0,2020-01-01,2.253928e-07,0.002139,POLYGON ((-74.10095170800003 4.574641781000025...,2020.0,train
3,3,1125506,60.0,0.0,0.0,NaN,1.0,0,1.0,0.0,3.0,NaN,NaN,0.0,3.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,2.600000e+08,Apartamento,venta,4.575924,-74.108039,365370.0,1402015.0,1800000.0,2020-01-01,2.724516e-06,0.009337,POLYGON ((-74.10817712599999 4.575135860999978...,2020.0,train
4,4,1204569,72.0,0.0,0.0,NaN,0.0,0,0.0,0.0,2.0,NaN,NaN,0.0,2.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1.0,0.0,1.250000e+08,Casa,venta,4.559000,-74.097000,368789.0,1408054.0,690000.0,2020-01-01,1.067251e-06,0.004753,POLYGON ((-74.09789949399999 4.559564367000007...,2020.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,48,726088,230.0,0.0,0.0,4.0,0.0,0,1.0,NaN,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.990000e+08,Casa,venta,4.737032,-74.038290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
49,49,752156,230.0,0.0,0.0,4.0,0.0,0,1.0,NaN,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.490000e+08,Casa,venta,4.735844,-74.038440,355194.0,8517015.0,2900000.0,2020-01-01,1.910642e-06,0.005839,POLYGON ((-74.03668588800002 4.735051208000016...,2020.0,test
50,50,1055205,230.0,0.0,0.0,4.0,0.0,0,1.0,NaN,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.490000e+08,Casa,venta,4.736058,-74.040634,390616.0,8519050.0,2400000.0,2020-01-01,5.794660e-07,0.003272,POLYGON ((-74.04008913899997 4.735252899999978...,2020.0,test
51,51,1289402,230.0,0.0,0.0,4.0,0.0,0,1.0,NaN,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,6.990000e+08,Casa,venta,4.737620,-74.040190,349138.0,8519048.0,2400000.0,2020-01-01,3.883588e-07,0.002686,POLYGON ((-74.03964318099997 4.737262713999996...,2020.0,test


In [ ]:
consol['tiempodeconstruido'] = consol['tiempodeconstruido'].replace('ntre 0 y 5 años', 'Entre 0 y 5 años')
consol['tiempodeconstruido'].unique()

array([nan, 'Entre 10 y 20 años', 'Entre 0 y 5 años', 'Entre 5 y 10 años',
       'Más de 20 años', '1 a 8 años', '16 a 30 años', '9 a 15 años',
       'Más de 30 años', 'Menos de 1 año', 'Remodelado'], dtype=object)

In [ ]:
apto = consol[consol.tipoinmueble=='Apartamento']
casa = consol[consol.tipoinmueble=='Casa']

,Unnamed: 0,Unnamed: 0.1,id,area,ascensor,balcon,banos,banoservicio,conjuntocerrado,cuartoservicio,deposito,estrato,estudio,garajes,gimnasio,habitaciones,halldealcobas,parqueaderovisitantes,piscina,piso,porteria,remodelado,saloncomunal,terraza,tiempodeconstruido,vigilancia,vista,zonalavanderia,valoradministracion,valorventa,tipoinmueble,tiponegocio,latitud,longitud,id2,MANCODIGO,V_REF,ANO,SHAPE_AREA,SHAPE_LEN,geometry,Ano2,tipo_data
64964,1277,1277,1016294,44.0,0.0,0.0,0.0,0.0,1,0.0,1.0,2.0,1.0,NaN,0.0,0.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.050000e+08,Casa,venta,4.517388,-74.093552,390532.0,1350008.0,360000.0,2020-01-01,8.137781e-07,0.004182,"POLYGON ((-74.09258214300002 4.51789359899999,...",2020.0,train
64965,1405,1405,256197,378.0,0.0,0.0,0.0,0.0,1,0.0,1.0,6.0,1.0,3.0,0.0,5.0,NaN,NaN,0.0,NaN,1.0,0.0,NaN,0.0,Entre 10 y 20 años,NaN,NaN,1.0,1367000.0,1.700000e+09,Casa,venta,4.730908,-74.077539,394147.0,9249005.0,740000.0,2020-01-01,4.020006e-06,0.011852,POLYGON ((-74.07793025900003 4.732228549000013...,2020.0,train
64966,1406,1406,281528,256.0,0.0,0.0,0.0,0.0,1,0.0,1.0,6.0,1.0,4.0,0.0,4.0,NaN,NaN,0.0,NaN,1.0,0.0,NaN,0.0,Entre 10 y 20 años,NaN,NaN,NaN,1050000.0,1.050000e+09,Casa,venta y arriendo,4.763322,-74.057870,353538.0,9113033.0,1150000.0,2020-01-01,3.132869e-06,0.007229,POLYGON ((-74.05853799499999 4.762050207000016...,2020.0,train
64967,1588,1588,980267,83.2,0.0,0.0,0.0,0.0,1,0.0,1.0,4.0,1.0,1.0,0.0,1.0,NaN,NaN,0.0,NaN,1.0,0.0,1.0,0.0,Entre 10 y 20 años,NaN,NaN,NaN,312000.0,4.600000e+08,Casa,venta,4.764850,-74.045840,382514.0,9133001.0,2460000.0,2020-01-01,2.570947e-06,0.006581,"POLYGON ((-74.044947928 4.763519453000015, -74...",2020.0,train
64968,1726,1726,73040,477.0,0.0,0.0,0.0,0.0,1,0.0,1.0,6.0,1.0,2.0,1.0,4.0,NaN,NaN,0.0,NaN,1.0,0.0,1.0,1.0,Entre 10 y 20 años,NaN,NaN,1.0,1827000.0,3.500000e+09,Casa,venta,4.662221,-74.046380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,48,48,726088,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.990000e+08,Casa,venta,4.737032,-74.038290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
49,49,49,752156,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.490000e+08,Casa,venta,4.735844,-74.038440,355194.0,8517015.0,2900000.0,2020-01-01,1.910642e-06,0.005839,POLYGON ((-74.03668588800002 4.735051208000016...,2020.0,test
50,50,50,1055205,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.490000e+08,Casa,venta,4.736058,-74.040634,390616.0,8519050.0,2400000.0,2020-01-01,5.794660e-07,0.003272,POLYGON ((-74.04008913899997 4.735252899999978...,2020.0,test
51,51,51,1289402,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,6.990000e+08,Casa,venta,4.737620,-74.040190,349138.0,8519048.0,2400000.0,2020-01-01,3.883588e-07,0.002686,POLYGON ((-74.03964318099997 4.737262713999996...,2020.0,test


In [ ]:
apto[['deposito', 'estudio']] = apto[['deposito', 'estudio']].fillna(0)

In [ ]:
casa[['deposito', 'estudio']]= casa[['deposito', 'estudio']].fillna(1)

In [ ]:
consol = apto.append(casa)

In [ ]:
cerrado = consol[consol.conjuntocerrado==1]
no_cerrado = consol[consol.conjuntocerrado==0]

In [ ]:
cerrado['porteria'] = cerrado['porteria'].fillna(1)
no_cerrado['porteria'] = no_cerrado['porteria'].fillna(0)

In [ ]:
consol= cerrado.append(no_cerrado)

      Unnamed: 0       id  ...    Ano2  tipo_data
335          335   363085  ...  2020.0      train
1387        1387   378687  ...     NaN      train
1392        1392   213992  ...     NaN      train
1401        1401   452566  ...  2020.0      train
1477        1477   676220  ...  2020.0      train
...          ...      ...  ...     ...        ...
48            48   726088  ...     NaN       test
49            49   752156  ...  2020.0       test
50            50  1055205  ...  2020.0       test
51            51  1289402  ...  2020.0       test
52            52   372796  ...  2020.0       test

[244543 rows x 42 columns]


In [ ]:
consol.loc[consol['valoradministracion']<10000, 'valoradministracion'] = float("NaN")

In [ ]:
train = consol[consol.tipo_data=="train"]
test = consol[consol.tipo_data=="test"]

        Unnamed: 0       id  ...    Ano2  tipo_data
335            335   363085  ...  2020.0      train
1387          1387   378687  ...     NaN      train
1392          1392   213992  ...     NaN      train
1401          1401   452566  ...  2020.0      train
1477          1477   676220  ...  2020.0      train
...            ...      ...  ...     ...        ...
244485      244485  6262941  ...     NaN      train
244486      244486  6262942  ...     NaN      train
244487      244487  6262943  ...  2020.0      train
244488      244488  6262944  ...  2020.0      train
244489      244489  6262945  ...  2020.0      train

[244490 rows x 42 columns]


In [ ]:
train.to_csv('/content/gdrive/MyDrive/shp/train_dataV03.csv')
test.to_csv('/content/gdrive/MyDrive/shp/test_dataV03.csv')

### Imputación via knn

In [ ]:
data1_train = pd.read_csv('/content/gdrive/MyDrive/shp/train_dataV03.csv')
data1_test = pd.read_csv('/content/gdrive/MyDrive/shp/test_dataV03.csv')
data1_train[['tipo_data']]= "train"
data1_test[['tipo_data']] = "test"

    Unnamed: 0  Unnamed: 0.1  ...    Ano2  tipo_data
0            0             0  ...  2020.0       test
1            1             1  ...     NaN       test
2            2             2  ...     NaN       test
3            3             3  ...     NaN       test
4            4             4  ...  2020.0       test
5            5             5  ...     NaN       test
6            6             6  ...     NaN       test
7            7             7  ...  2020.0       test
8            8             8  ...  2020.0       test
9            9             9  ...  2020.0       test
10          10            10  ...  2020.0       test
11          11            11  ...     NaN       test
12          12            12  ...  2020.0       test
13          13            13  ...     NaN       test
14          14            14  ...     NaN       test
15          15            15  ...  2020.0       test
16          16            16  ...     NaN       test
17          17            17  ...     NaN     

In [ ]:
from sklearn.impute import KNNImputer
#https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e
K = 5 #CHICASSS hay que escoger ese casa en base a alguna medida de error :c 
# mientras tanto 5 casas a la redonda xd
imputer = KNNImputer(n_neighbors = 5)

In [ ]:
consol1 = data1_train.append(data1_test)
consol1

,Unnamed: 0,Unnamed: 0.1,id,area,ascensor,balcon,banos,banoservicio,conjuntocerrado,cuartoservicio,deposito,estrato,estudio,garajes,gimnasio,habitaciones,halldealcobas,parqueaderovisitantes,piscina,piso,porteria,remodelado,saloncomunal,terraza,tiempodeconstruido,vigilancia,vista,zonalavanderia,valoradministracion,valorventa,tipoinmueble,tiponegocio,latitud,longitud,id2,MANCODIGO,V_REF,ANO,SHAPE_AREA,SHAPE_LEN,geometry,Ano2,tipo_data
0,335,335,363085,NaN,0.0,0.0,NaN,0.0,1,0.0,0.0,2.0,0.0,NaN,0.0,2.0,NaN,NaN,0.0,NaN,1.0,0.0,NaN,0.0,Entre 0 y 5 años,NaN,NaN,NaN,55000.0,1.160000e+08,Apartamento,venta,4.651098,-74.168666,380234.0,6518014.0,1850000.0,2020-01-01,1.935570e-06,0.005504,"POLYGON ((-74.168539409 4.650990077000017, -74...",2020.0,train
1,1387,1387,378687,0.0,1.0,0.0,0.0,0.0,1,1.0,0.0,6.0,1.0,0.0,0.0,4.0,NaN,NaN,0.0,1.0,1.0,0.0,NaN,0.0,Entre 10 y 20 años,NaN,Exterior,1.0,NaN,1.590000e+09,Apartamento,venta,4.690135,-74.049683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
2,1392,1392,213992,345.0,0.0,0.0,0.0,0.0,1,0.0,0.0,6.0,1.0,2.0,1.0,3.0,NaN,NaN,0.0,NaN,1.0,1.0,1.0,0.0,Entre 10 y 20 años,NaN,NaN,1.0,1187000.0,1.800000e+09,Apartamento,venta,4.697660,-74.052653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
3,1401,1401,452566,80.0,0.0,0.0,0.0,0.0,1,0.0,0.0,NaN,1.0,1.0,0.0,3.0,NaN,NaN,0.0,NaN,1.0,0.0,NaN,0.0,Entre 0 y 5 años,NaN,NaN,NaN,190000.0,3.400000e+08,Apartamento,venta,4.731135,-74.054373,352340.0,9104021.0,2900000.0,2020-01-01,3.127788e-06,0.013924,POLYGON ((-74.05485601800001 4.731215995000014...,2020.0,train
4,1477,1477,676220,87.0,0.0,0.0,0.0,0.0,1,0.0,0.0,4.0,1.0,1.0,1.0,0.0,NaN,1.0,1.0,9.0,1.0,0.0,NaN,0.0,1 a 8 años,NaN,NaN,NaN,200000.0,4.200000e+08,Apartamento,venta,4.642000,-74.113998,394636.0,6306002.0,2000000.0,2020-01-01,8.635009e-06,0.015905,POLYGON ((-74.11382247699999 4.643146832000014...,2020.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,48,48,726088,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.990000e+08,Casa,venta,4.737032,-74.038290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test
49,49,49,752156,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.490000e+08,Casa,venta,4.735844,-74.038440,355194.0,8517015.0,2900000.0,2020-01-01,1.910642e-06,0.005839,POLYGON ((-74.03668588800002 4.735051208000016...,2020.0,test
50,50,50,1055205,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,7.490000e+08,Casa,venta,4.736058,-74.040634,390616.0,8519050.0,2400000.0,2020-01-01,5.794660e-07,0.003272,POLYGON ((-74.04008913899997 4.735252899999978...,2020.0,test
51,51,51,1289402,230.0,0.0,0.0,4.0,0.0,0,1.0,1.0,4.0,1.0,2.0,0.0,4.0,1.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,Más de 20 años,NaN,NaN,1.0,NaN,6.990000e+08,Casa,venta,4.737620,-74.040190,349138.0,8519048.0,2400000.0,2020-01-01,3.883588e-07,0.002686,POLYGON ((-74.03964318099997 4.737262713999996...,2020.0,test


In [ ]:
consol1.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'area', 'ascensor', 'balcon',
       'banos', 'banoservicio', 'conjuntocerrado', 'cuartoservicio',
       'deposito', 'estrato', 'estudio', 'garajes', 'gimnasio', 'habitaciones',
       'halldealcobas', 'parqueaderovisitantes', 'piscina', 'piso', 'porteria',
       'remodelado', 'saloncomunal', 'terraza', 'tiempodeconstruido',
       'vigilancia', 'vista', 'zonalavanderia', 'valoradministracion',
       'valorventa', 'tipoinmueble', 'tiponegocio', 'latitud', 'longitud',
       'id2', 'MANCODIGO', 'V_REF', 'ANO', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry', 'Ano2', 'tipo_data'],
      dtype='object')

In [ ]:
#area knn
#banos knn
#estrato knn
#garajes knn
#habitaciones knn
#piso knn
#salon comunal knn
#vigilancia knn
#zonalavanderia casas = 1, conjunto residencial_1 = knn, resto
#hall de alcobas condicional al numero de cuartos >1
#parqueaderovisitantes condicional al numero de garajes>1 = garajes - 1

In [ ]:
area = consol1.loc[:,['area','longitud','latitud']].values
df_area = pd.DataFrame(imputer.fit_transform(area),columns = [[ 'area','longitud','latitud']])

In [ ]:
df_area = pd.DataFrame(imputer.fit_transform(area),columns = [[ 'area','longitud','latitud']])

In [ ]:
banos = consol1.loc[:,['banos','longitud','latitud']].values
df_banos = pd.DataFrame(imputer.fit_transform(banos),columns = [[ 'banos','longitud','latitud']])
df_banos.loc[:, ['banos']] = np.round(df_banos.loc[:, ['banos']] )

In [ ]:
estrato = consol1.loc[:,['estrato','longitud','latitud']].values
df_estrato = pd.DataFrame(imputer.fit_transform(estrato),columns = [[ 'estrato','longitud','latitud']])
df_estrato.loc[:, ['estrato']] = np.round(df_estrato.loc[:, ['estrato']] )

In [ ]:
garajes = consol1.loc[:,['garajes','longitud','latitud']].values
df_garajes = pd.DataFrame(imputer.fit_transform(garajes),columns = [[ 'garajes','longitud','latitud']])
df_garajes.loc[:, ['garajes']] = np.round(df_garajes.loc[:, ['garajes']] )

In [ ]:
piso = consol1.loc[:,['piso','longitud','latitud']].values
df_piso = pd.DataFrame(imputer.fit_transform(piso),columns = [[ 'piso','longitud','latitud']])
df_piso.loc[:, ['piso']] = np.round(df_piso.loc[:, ['piso']] )

In [ ]:
habitaciones = consol1.loc[:,['habitaciones','longitud','latitud']].values
df_habitaciones = pd.DataFrame(imputer.fit_transform(habitaciones),columns = [[ 'habitaciones','longitud','latitud']])
df_habitaciones.loc[:, ['habitaciones']] = np.round(df_habitaciones.loc[:, ['habitaciones']] )

In [ ]:
saloncomunal = consol1.loc[:,['saloncomunal','longitud','latitud']].values
df_saloncomunal = pd.DataFrame(imputer.fit_transform(saloncomunal),columns = [[ 'saloncomunal','longitud','latitud']])
df_saloncomunal.loc[:, ['saloncomunal']] = np.round(df_saloncomunal.loc[:, ['saloncomunal']] )

In [ ]:
vigilancia = consol1.loc[:,['vigilancia','longitud','latitud']].values
df_vigilancia = pd.DataFrame(imputer.fit_transform(vigilancia),columns = [[ 'vigilancia','longitud','latitud']])
df_vigilancia.loc[:, ['vigilancia']] = np.round(df_vigilancia.loc[:, ['vigilancia']] )

In [ ]:
apto1 = consol1[consol1.tipoinmueble=='Apartamento']      
casa1 = consol1[consol1.tipoinmueble=='Casa']

In [ ]:
casa1[['zonalavanderia']]= casa1[['zonalavanderia']].fillna(1)
consol1 = apto1.append(casa1)

In [ ]:
zonalavanderia = consol1.loc[:,['zonalavanderia','longitud','latitud']].values
df_zonalavanderia = pd.DataFrame(imputer.fit_transform(zonalavanderia),columns = [[ 'zonalavanderia','longitud','latitud']])
df_zonalavanderia.loc[:, ['zonalavanderia']] = np.round(df_zonalavanderia.loc[:, ['zonalavanderia']] )

In [ ]:
habi_menor_1 = consol1.loc[consol1.loc[ : , 'habitaciones'] <= 1]
habi_mayor_1 = consol1.loc[consol1.loc[ : , 'habitaciones']  > 1]

In [ ]:
habi_menor_1[['halldealcobas']] = habi_menor_1[['halldealcobas']].fillna(0)
consol1= habi_menor_1.append(habi_mayor_1)

In [ ]:
halldealcobas = consol1.loc[: , ['halldealcobas','longitud','latitud']].values
df_halldealcobas = pd.DataFrame(imputer.fit_transform(halldealcobas),columns = [[ 'halldealcobas','longitud','latitud']])
df_halldealcobas.loc[:, ['halldealcobas']] = np.round(df_halldealcobas.loc[:, ['halldealcobas']] )

In [ ]:
parqueaderovisitantes = consol1.loc[: , ['parqueaderovisitantes','longitud','latitud']].values
df_parqueaderovisitantes = pd.DataFrame(imputer.fit_transform(parqueaderovisitantes),columns = [[ 'parqueaderovisitantes','longitud','latitud']])
df_parqueaderovisitantes.loc[:, ['parqueaderovisitantes']] = np.round(df_parqueaderovisitantes.loc[:, ['parqueaderovisitantes']] )

In [ ]:
valoradministracion = consol1.loc[: , ['valoradministracion','longitud','latitud']].values
df_valoradministracion = pd.DataFrame(imputer.fit_transform(valoradministracion),columns = [[ 'valoradministracion','longitud','latitud']])
df_valoradministracion.loc[:, ['valoradministracion']] = np.round(df_valoradministracion.loc[:, ['valoradministracion']] )

In [ ]:
consol1[['area']] = df_area[['area']]
consol1[['banos']] = df_banos[['banos']]
consol1[['estrato']] = df_estrato[['estrato']]
consol1[['garajes']] = df_garajes[['garajes']]
consol1[['habitaciones']] = df_habitaciones[['habitaciones']]
consol1[['piso']] = df_piso[['piso']]
consol1[['saloncomunal']] = df_saloncomunal[['saloncomunal']]
consol1[['vigilancia']] = df_vigilancia[['vigilancia']]
consol1[['zonalavanderia']] = df_zonalavanderia[['zonalavanderia']]
consol1[['halldealcobas']] = df_halldealcobas[['halldealcobas']]
consol1[['parqueaderovisitantes']] = df_parqueaderovisitantes[['parqueaderovisitantes']]
consol1[['valoradministracion']] = df_valoradministracion[['valoradministracion']]

NameError: ignored

In [ ]:
consol1.isna().sum()

Unnamed: 0                    0
Unnamed: 0.1                  0
Unnamed: 0.1.1                0
id                            0
area                          0
ascensor                      0
balcon                        0
banos                         0
banoservicio                  0
conjuntocerrado               0
cuartoservicio                0
deposito                      0
estrato                       0
estudio                       0
garajes                       0
gimnasio                      0
habitaciones                  0
halldealcobas                 0
parqueaderovisitantes         0
piscina                       0
piso                          0
porteria                      0
remodelado                    0
saloncomunal                  0
terraza                       0
tiempodeconstruido        60578
vigilancia                    0
vista                    142409
zonalavanderia                0
valoradministracion           0
valorventa                    0
tipoinmu

In [ ]:
train1 = consol1[consol1.tipo_data=="train"]
test1 = consol1[consol1.tipo_data=="test"]

In [ ]:
train1.to_csv('/content/gdrive/MyDrive/shp/train_dataV04.csv')
test1.to_csv('/content/gdrive/MyDrive/shp/test_dataV04.csv')

## Correlaciones lineales entre las variables

In [ ]:
# Gráfica interactiva de las diferentes variables en la tabla data
df = data
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index,y = df[df.columns[0]],visible = True))
updatemenu = []
buttons = []
for col in df.columns:
    buttons.append(dict(method = 'restyle',label = col,visible = True, 
                        args = [{'y':[df[col]],'x':[df.index],'type':'scatter'}, [0]],))
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)
updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['x'] = 0.7
updatemenu[0]['y'] = 1.15
updatemenu[0]['showactive'] = True
fig.update_layout(showlegend = False,updatemenus = updatemenu)
fig.update_layout(yaxis = dict(showexponent = 'all',exponentformat = 'e'))
fig.update_layout(xaxis_title = XLABEL, yaxis_title = YLABEL,template = 'plotly_white')
fig.show()

In [ ]:
#Calcular correlación entre variables
df = df[data.index <= train_date[1]]
correlation_Pearson = df.corr(method='pearson')
i = np.argsort(correlation_Pearson.iloc[0,:]).values
correlation_Pearson = df.iloc[:,i].corr(method='pearson')
correlation_Kendall = df.iloc[:,i].corr(method='kendall')
correlation_Spearman = df.iloc[:,i].corr(method='spearman')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=correlation_Pearson.values,
                         x=np.arange(0,correlation_Pearson.shape[0]),
                         y=correlation_Pearson.columns,hoverongaps = False))
updatemenu = []
buttons = []
buttons.append(dict(method = 'restyle',label = "Correlación de pearson",visible = True, 
                        args = [{'z':[correlation_Pearson.values],
                                 'x':[np.arange(0,correlation_Pearson.shape[0])],
                                 'y':[correlation_Pearson.columns],
                                 'type':'heatmap'}, [0]],))
buttons.append(dict(method = 'restyle',label = "Correlación de kendall",visible = True, 
                        args = [{'z':[correlation_Kendall.values],
                                 'x':[np.arange(0,correlation_Kendall.shape[0])],
                                 'y':[correlation_Kendall.columns],
                                 'type':'heatmap'}, [0]],))
buttons.append(dict(method = 'restyle',label = "Correlación de spearman",visible = True, 
                        args = [{'z':[correlation_Spearman.values],
                                 'x':[np.arange(0,correlation_Spearman.shape[0])],
                                 'y':[correlation_Spearman.columns],
                                 'type':'heatmap'}, [0]],))
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)
updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['x'] = 0.7
updatemenu[0]['y'] = 1.15
updatemenu[0]['showactive'] = True
fig.update_xaxes(side="top")
fig.update_layout(
    showlegend = False,
    height=500,width = 1000,
    updatemenus = updatemenu,
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0,correlation_Pearson.shape[0]),
        ticktext = np.arange(0,correlation_Pearson.shape[0])[::-1]
    ),
    legend_orientation="h"
)
fig.show()

In [ ]:
correlations_Tab = pd.DataFrame({'Pearson':correlation_Pearson[VARIABLE_EXCEL].values[::-1],
                                'Kendall':correlation_Kendall[VARIABLE_EXCEL].values[::-1],
                                'Spearman':correlation_Spearman[VARIABLE_EXCEL].values[::-1]})
correlations_Tab.index = correlation_Pearson.index[::-1]
# Tabla de resultados ordenados de acuerdo a la correlación de Pearson
## Marcamos de color verde aquellas correlaciones mayores a 0.4
correlations_Tab.style.applymap(_color_red_or_green)

# Validación cruzada

In [ ]:
Ny = int(Y[Y.index <= train_date[1]].shape[0]) 
Nk = int(Y[Y.index <= train_date[1]].shape[0]/2)

In [ ]:
# Función para validaciones con parámetros: n (cantidad de datos),min_n (tamaños de lotes)
def split(n,min_n = 3):
     cc = min_n
     min = 0
     while cc != n:
       idx = np.arange(0,cc, dtype=int)
       if (not Rolling):idx = np.arange(min,cc, dtype=int)
       if (cc == n):break
       idy = np.arange(cc,cc + 1, dtype=int)
       if (not Rolling):idy = np.arange(cc,n, dtype=int)
       cc = cc + 1
       min = min + 1
       yield idx, idy
for train, test in split(min_n = Nk,n = Ny):
    print("%s %s" % (train, test))

# Calibración hiperparámetros

In [ ]:
params_i={}
  iter_max=10000
  param = [{'kernel': ['rbf'], #Los hiperparámetros para este kernel son gamma y C
            'gamma':np.arange(start=0.0001, stop=10, step=2),
            'C':np.arange(start=0.0001, stop=5, step=1),},
          {'kernel': ['poly'], #Los hiperparámetros para este kernerl son gammas, coef0 (intercepto), degree (grados polinomio) y C
            'gamma': np.arange(start=0.0001, stop=10, step=2),
            'coef0': np.arange(start=0.0001, stop=5, step=1), 
            'degree': (2, 3 , 4),
            'C':np.arange(start=0.0001, stop=5, step=1),},
          {'kernel': ['sigmoid'], #Los hiperparámetros para este kernel son coef0 (intercepto), C y gamma
          'coef0':np.arange(start=0.0001, stop=5, step=1),
          'C':np.arange(start=0.0001, stop=5, step=1),
          'gamma':np.arange(start=0.0001, stop=10, step=2)}]
  model_svm = SVR(max_iter = iter_max,epsilon = 0.01)
  for score in scoring:
    kfold = split(min_n = Nk,n = Ny)
    grid_i = GridSearchCV(model_svm, param, cv=kfold, scoring=score)
    grid_i.fit(data_base['X']['rescaled'], data_base['Y']['rescaled'][:,0])
    par_1 =  grid_i.best_params_['gamma']
    params_i['gamma'] = np.arange(start=starMV(par_1 - 0.1*10), stop=par_1 + 0.1*10, step=0.1)
    par_2 =  grid_i.best_params_['C']
    params_i['C'] = np.arange(start=starMV(par_2 - 0.1*10), stop=par_2 + 0.1*10, step=0.1)
    model_svm = SVR(max_iter=iter_max,epsilon = 0.01,kernel='rbf')
    kernel_i = grid_i.best_params_['kernel']
    if kernel_i == 'sigmoid':
      par_3 =  grid_i.best_params_['coef0']
      params_i['coef0'] = np.arange(start=starMV(par_3 - 0.1*10), stop=par_3 + 0.1*10, step=0.1)
      model_svm = SVR(max_iter=iter_max,epsilon = 0.01,kernel='sigmoid')
    if kernel_i == 'poly':
      par_3 =  grid_i.best_params_['coef0']
      params_i['coef0'] = np.arange(start=starMV(par_3 - 0.1*10), stop=par_3 + 0.1*10, step=0.1)
      degree_i = grid_i.best_params_['degree']
      model_svm = SVR(max_iter=iter_max,epsilon = 0.01,kernel='poly',degree=degree_i)
    kfold = split(min_n = Nk,n = Ny)
    grid_i = GridSearchCV(model_svm, params_i, cv=kfold, scoring=score)
    grid_i.fit(data_base['X']['rescaled'], data_base['Y']['rescaled'][:,0])
    grids['SVM' + score] = grid_i.best_params_
    grids['SVM' + score]['kernel'] = kernel_i
    if kernel_i == 'poly':
      grids['SVM' + score]['degree'] = degree_i
    print (score, grids['SVM' + score])

In [ ]:
paramMLP = {'solver':['adam'],
          'activation':['logistic','tanh','relu'],
          'hidden_layer_sizes': [(2),(4),(8),(2,4),(4,3),(4,8),(8,2),(4,3,2)],
          'alpha': [0.01,0.001,0.0001,0.00001],
          'learning_rate_init':[0.001,0.01,0.0001,0.1],
          'beta_1':[0.3,0.6,0.9,0.99,0.999],
          'beta_2':[0.3,0.6,0.9,0.99,0.999]}
  modelMLP= MLPRegressor(batch_size='auto',verbose=False,shuffle=False,max_iter=1000,random_state=7,validation_fraction=0)
  best_params=[]
  for score in scoring:
    kfold = split(min_n = Nk,n = Ny)
    gridMLP = GridSearchCV(modelMLP, paramMLP, cv=kfold, scoring=score,n_jobs=-1)
    gridMLP.fit(data_base['X']['rescaled'], data_base['Y']['rescaled'][:,0])
    best= score, gridMLP.best_params_
    best_params.append(best)
    grids['MLP' + score] = gridMLP.best_params_
    print(score, gridMLP.best_params_)

In [ ]:
#Descomentar para guardar nueva grilla
#saved = joblib.dump(grids, os.path.join('Grids Consumo Carbon Termico.joblib'))
#Carga los parámetros elegidos para ahorrar tiempo de ejecución
grids = joblib.load('Grids Consumo Carbon Termico.joblib')

# Modelos finales

In [ ]:
 #Función para iniciazar todos los modelos a competir
def inicializarModelos(criterio):
  modelos = []
  grids_i = grids
  ## Modelo SVR final del script sin modificar
  if grids_i['SVM' + criterio]['kernel'] == 'poly':
    modelos.append(('SVM', SVR(max_iter=1000,kernel = 'poly',
                               coef0 =  grids_i['SVM' + criterio]['coef0'],
                               gamma = grids_i['SVM' + criterio]['gamma'],
                               C = grids_i['SVM' + criterio]['C'],
                               degree = grids_i['SVM' + criterio]['degree'],
                                 epsilon = 0.001)))
  if grids_i['SVM' + criterio]['kernel'] =='rbf':
    modelos.append(('SVM', SVR(max_iter=1000,kernel = 'rbf',
                               gamma = grids_i['SVM' + criterio]['gamma'],
                               C = grids_i['SVM' + criterio]['C'],
                               epsilon = 0.001)))
  if grids_i['SVM' + criterio]['kernel'] =='sigmoid':
    modelos.append(('SVM', SVR(max_iter = 1000,kernel = 'sigmoid',
                               coef0 = grids_i['SVM' + criterio]['coef0'],
                                 gamma = grids_i['SVM' + criterio]['gamma'],
                                 C = grids_i['SVM' + criterio]['C'],
                                 epsilon = 0.001)))
  modelos.append(('MLP',skl.neural_network.MLPRegressor(hidden_layer_sizes = grids_i['MLP' + criterio]['hidden_layer_sizes'],
                                                          activation = grids_i['MLP' + criterio]['activation'],
                                                          solver = grids_i['MLP' + criterio]['solver'],
                                                          alpha = grids_i['MLP' + criterio]['alpha'],
                                                          beta_1 = grids_i['MLP' + criterio]['beta_1'],
                                                          beta_2 = grids_i['MLP' + criterio]['beta_2'],
                                                          learning_rate_init = grids_i['MLP' + criterio]['learning_rate_init'],
                                                          verbose = False,random_state = 7,
                                                          validation_fraction = 0,shuffle = False,max_iter = 1000)))
  modelos.append(('DecisionTree',DecisionTreeRegressor(splitter = 'best',
                                                              min_samples_leaf = 2,
                                                              max_depth = grids_i['DecisionTree' + criterio]['max_depth'],
                                                              random_state = 7
                                                              )))
  return modelos

In [ ]:
# Iniciamos los modelos
Modelos = {}
for j in scoring:
  Modelos[j] = inicializarModelos(criterio = j)
Resultados = {} # Lista para guardar predicciones y medidas de ajuste
for j in scoring:
  Resultados[j] = {}
  results_cv = []
  names_cv = []
  predictions = {}
  for name, modelo in Modelos[j]:
    # Calculamos las medidas de perdida
    kfold = split(min_n = 7,n = 14)
    cv = cross_val_score(modelo,data_base['X']['rescaled'], data_base['Y']['rescaled'][:,0],
                         cv = kfold, scoring = j)
    # Ajustamos cada modelo
    modelo.fit(data_base['X']['rescaled'][0:sum(Y.index <= train_date[1])],
              data_base['Y']['rescaled'][:,0][0:sum(Y.index <= train_date[1])])
    # Realizamos las predicciones
    predictions[name] = modelo.predict(data_base['rescaled'])
    results_cv.append(cv)
    names_cv.append(np.repeat(name,cv.shape[0]))
  Resultados[j]['predictions'] = predictions
  Resultados[j]['cv'] = {}
  Resultados[j]['cv']['value'] = results_cv
  Resultados[j]['cv']['names'] = names_cv

In [ ]:
# Modelos para el promedio
Modelos_promedio = ['Lasso','ElasticNet','SVM','MLP','DecisionTree']

In [ ]:
for j in scoring:
  predictions = Resultados[j]['predictions']
  scalerY = data_base['Y']['scaler']
  dfResModels =  pd.DataFrame({'Lasso':INVT(predictions['Lasso'],scalerY),
                            'ElasticNet':INVT(predictions['ElasticNet'],scalerY),
                            'SVM':INVT(predictions['SVM'],scalerY),
                            'MLP':INVT(predictions['MLP'],scalerY),
                            'DecisionTree':INVT(predictions['DecisionTree'],scalerY)})
  dfResModels['medias'] = np.nan
  dfResModels['medias'][Ny:]=dfResModels[Modelos_promedio][Ny:].mean(axis=1)
  dfResModels['Real']=Y.values[Y.index<= prediction_data[1]]
  dfResModels.index = X.index[Y.index<= prediction_data[1]]
  Resultados[j]['dataframe'] = dfResModels

In [ ]:
# Graficar resultado de la proyección
df = Resultados[scoring[0]]['dataframe']
fig = go.Figure()
for i in df.columns:
  fig.add_scatter(x=df.index, y=df[i],mode='lines',name=i)
fig.update_layout(title_text="Predicciones modelos")
fig.update_layout(yaxis = dict(showexponent = 'all',exponentformat = 'e'))
fig.update_layout(xaxis_title = XLABEL, yaxis_title = YLABEL,template = "plotly_white")
fig.show()

In [ ]:
sc = scoring[0]
df = pd.DataFrame({'values': np.reshape(np.array(Resultados[sc]['cv']['value']),-1),
                   'model': np.reshape(np.array(Resultados[sc]['cv']['names']),-1)})
fig = go.Figure()
fig.add_trace(go.Box(y = df['values'],x = df['model'],boxpoints = 'all',boxmean=True))
fig.update_layout(boxmode='group',template = "plotly_white",title_text="Cv " + sc)
fig.show()

In [ ]:
df.groupby('model').mean().sort_values('values',ascending=False).T

In [ ]:
models=Resultados[scoring[0]]['dataframe'].columns
MSE=[]
for i in models[:-2]:
  aux=mean_squared_error(data_base['Y']['rescaled'],Resultados[scoring[0]]['predictions'][i][:Ny])
  MSE.append(aux)
b=pd.DataFrame({'MSE':MSE},index=models[:-2]) 
b.sort_values('MSE',ascending=True).T